In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/results/model_last.hdf5
/kaggle/input/results/best_model.hdf5
/kaggle/input/results/submission.csv
/kaggle/input/results/car/submission1.csv
/kaggle/input/sf-dl-car-classification/train.zip
/kaggle/input/sf-dl-car-classification/sample-submission.csv
/kaggle/input/sf-dl-car-classification/test.zip
/kaggle/input/sf-dl-car-classification/train.csv
/kaggle/input/efficientnet-keras-noisystudent-weights-b0b7/efficientnet-b2_noisy-student_notop.h5
/kaggle/input/efficientnet-keras-noisystudent-weights-b0b7/efficientnet-b1_noisy-student_notop.h5
/kaggle/input/efficientnet-keras-noisystudent-weights-b0b7/efficientnet-b3_noisy-student_notop.h5
/kaggle/input/efficientnet-keras-noisystudent-weights-b0b7/efficientnet-b7_noisy-student_notop.h5
/kaggle/input/efficientnet-keras-noisystudent-weights-b0b7/efficientnet-b0_noisy-student_notop.h5
/kaggle/input/efficientnet-keras-noisystudent-weights-b0b7/efficientnet-b6_noisy-student_notop.h5
/kaggle/input/efficientnet-keras-noisystudent-weig

In [2]:
!nvidia-smi

Mon May 17 19:56:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.06    Driver Version: 450.51.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import zipfile
import csv
import sys
import os


import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers

from tensorflow.keras.models import Model
import tensorflow.keras.models as Model
from tensorflow.keras.layers import *
import tensorflow.keras.layers as layers

from tensorflow.keras.applications.xception import Xception


from sklearn.model_selection import train_test_split, StratifiedKFold

import PIL
from PIL import ImageOps, ImageFilter

#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5

#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

print(os.listdir("../input"))
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)
print('Keras        :', tf.keras.__version__)

['results', 'sf-dl-car-classification', 'efficientnet-keras-noisystudent-weights-b0b7', 'model-base']
Python       : 3.7.10 | packaged by conda-forge | (default, Feb 19 2021, 16:07:37) 
Numpy        : 1.19.5
Tensorflow   : 2.4.1
Keras        : 2.4.0


In [4]:
# В setup выносим основные настройки: так удобнее их перебирать в дальнейшем.

EPOCHS               = 10  # эпох на обучение
BATCH_SIZE           = 16 # уменьшаем batch если сеть большая, иначе не влезет в память на GPU
LR                   = 1e-4
VAL_SPLIT            = 0.15 # сколько данных выделяем на тест = 15%

CLASS_NUM            = 10  # количество классов в нашей задаче
IMG_SIZE             = 260 # какого размера подаем изображения в сеть
IMG_CHANNELS         = 3   # у RGB 3 канала
input_shape          = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)

DATA_PATH = '../input/sf-dl-car-classification/'
PATH = "../working/car/" # рабочая директория
RANDOM_SEED = 42
ratio = 640/480

In [5]:
print('Распаковываем картинки')
# Will unzip the files so that you can see them..
for data_zip in ['train.zip', 'test.zip']:
    with zipfile.ZipFile(DATA_PATH+data_zip,"r") as z:
        z.extractall(PATH)
        
print(os.listdir(PATH))

Распаковываем картинки
['test_upload', 'train']


In [6]:
#посмотрим пример картинок
print('Пример картинок')
plt.figure(figsize=(12,8))

random_image = train_df.sample(n=9)
random_image_paths = random_image['Id'].values
random_image_cat = random_image['Category'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(DATA_PATH+f'train/train/{random_image_cat[index]}/{path}')
    plt.subplot(3,3, index+1)
    plt.imshow(im)
    plt.title('Class: '+str(random_image_cat[index]))
    plt.axis('off')
plt.show()

Пример картинок


NameError: name 'train_df' is not defined

<Figure size 864x576 with 0 Axes>

In [7]:
#посмотрим на размер картинок
image = PIL.Image.open(DATA_PATH+f'train/train/0/100380.jpg')
imgplot = plt.imshow(image)
plt.show()
image.size

FileNotFoundError: [Errno 2] No such file or directory: '../input/sf-dl-car-classification/train/train/0/100380.jpg'

In [8]:
# код для копированич модели обученной ранее
# from distutils.dir_util import copy_tree
# copy_tree('../input/results/', PATH)

In [9]:
train_df = pd.read_csv(DATA_PATH+"train.csv")
sample_submission = pd.read_csv(DATA_PATH+"sample-submission.csv")
train_df.head()

,Id,Category
0,100155.jpg,0
1,100306.jpg,0
2,100379.jpg,0
3,100380.jpg,0
4,100389.jpg,0


In [10]:
# воспользуемся библиотекой для аугументации изображений
! pip install git+https://github.com/mjkvaak/ImageDataAugmentor

  Cloning https://github.com/mjkvaak/ImageDataAugmentor to /tmp/pip-req-build-k1ez2dug
  Running command git clone -q https://github.com/mjkvaak/ImageDataAugmentor /tmp/pip-req-build-k1ez2dug
  Created wheel for ImageDataAugmentor: filename=ImageDataAugmentor-0.0.0-py3-none-any.whl size=29531 sha256=c9a50ff95867a6b6a95cb1fc67e63e1ecf595af9679696e527b8689bba375f5c
  Stored in directory: /tmp/pip-ephem-wheel-cache-_41y9xwu/wheels/c9/bd/73/9cfa59d2393dae55bbcc30f5aa901f55fe531c66efebbc8fc3
Successfully built ImageDataAugmentor


In [11]:
import tensorflow as tf
from ImageDataAugmentor.image_data_augmentor import *
import albumentations

In [12]:

AUGMENTATIONS = albumentations.Compose([
    albumentations.HorizontalFlip(p=0.5),
    albumentations.Rotate(limit=30, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=False, p=0.5),
    albumentations.OneOf([
        albumentations.CenterCrop(height=224, width=200),
        albumentations.CenterCrop(height=200, width=224),
    ],p=0.5),
    albumentations.OneOf([
        albumentations.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3),
        albumentations.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1)
    ],p=0.5),
    albumentations.GaussianBlur(p=0.05),
    albumentations.HueSaturationValue(p=0.5),
    albumentations.RGBShift(p=0.5),
    albumentations.FancyPCA(alpha=0.1, always_apply=False, p=0.5),
    albumentations.Resize(240, 320)
])

# dataloaders
train_datagen_2 = ImageDataAugmentor(
        rescale=1./255,
        augment=AUGMENTATIONS,
        validation_split=VAL_SPLIT)

test_datagen = ImageDataGenerator(rescale=1. / 255)

# data generators
train_generator_2 = train_datagen_2.flow_from_directory(
        PATH+'train/',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True, 
        seed=RANDOM_SEED,
        subset='training')

test_generator_2 = train_datagen_2.flow_from_directory(
        PATH+'train/',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True, seed=RANDOM_SEED,
        subset='validation')

test_sub_generator = test_datagen.flow_from_dataframe( 
    dataframe=sample_submission,
    directory=PATH+'test_upload/',
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,)

/opt/conda/lib/python3.7/site-packages/ImageDataAugmentor/image_data_augmentor.py:332: UserWarning: Passing `seed` in `.flow_from_directory` has been been removed: pass  `seed` as parameter in `ImageDataAugmentor(..., seed=...)` instead
  warnings.warn('Passing `seed` in `.flow_from_directory` has been been removed: pass  `seed` '


Found 13232 images belonging to 10 classes.
Found 2329 images belonging to 10 classes.
Found 6675 validated image filenames.


In [13]:
!pip install efficientnet

     |████████████████████████████████| 50 kB 2.6 MB/s 


In [14]:
import efficientnet.tfkeras as efn

In [15]:
# загрузим модель Efficientnet-b2 c весами noisy student
model_path = '../input/efficientnet-keras-noisystudent-weights-b0b7/efficientnet-b5_noisy-student_notop.h5'
base_model = efn.EfficientNetB5(weights=model_path, include_top=False, input_shape=input_shape)

In [16]:
base_model.trainable = False

In [17]:
# instantiating the model in the strategy scope creates the model on the TPU
#with tpu_strategy.scope():
# steps_per_execution=32

model2 = Model.Sequential()
model2.add(base_model)
# model2.add(layers.Conv2D(32, 3, activation = 'relu', padding = 'same', 
#                         input_shape = input_shape))
# model2.add(layers.BatchNormalization())

model2.add(layers.GlobalAveragePooling2D())
model2.add(layers.BatchNormalization()) 
#model2.add(layers.Dropout(0.25))
   
model2.add(layers.Dense(256, activation='relu'))
model2.add(layers.Dropout(0.5))

#model2.add(layers.BatchNormalization())
#     model2.add(layers.Dropout(0.25))

model2.add(layers.Dense(CLASS_NUM, activation='softmax'))

model2.compile(loss="categorical_crossentropy", 
               optimizer=optimizers.Adam(lr=LR), 
               metrics=["accuracy"],
               )

In [18]:
checkpoint = ModelCheckpoint('best_model.hdf5', 
                             monitor = ['val_accuracy'], 
                             verbose = 1, 
                             mode = 'max')

earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                             patience=3, 
                                             restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25,
                              patience=2, min_lr=0.0000001, verbose=1,
                             mode='auto')

callbacks_list = [checkpoint,earlystop,reduce_lr] # lrscheduler,lr_callback

In [19]:
history = model2.fit(
        train_generator_2,
        steps_per_epoch = train_generator_2.samples//train_generator_2.batch_size,
        validation_data = test_generator_2, 
        validation_steps = test_generator_2.samples//test_generator_2.batch_size,
        epochs = EPOCHS,
        callbacks = callbacks_list
)


Epoch 1/10
827/827 [==============================] - 211s 236ms/step - loss: 2.6069 - accuracy: 0.2222 - val_loss: 1.4819 - val_accuracy: 0.4651

Epoch 00001: saving model to best_model.hdf5
Epoch 2/10
827/827 [==============================] - 195s 236ms/step - loss: 1.9808 - accuracy: 0.3332 - val_loss: 1.3443 - val_accuracy: 0.5250

Epoch 00002: saving model to best_model.hdf5
Epoch 3/10
827/827 [==============================] - 199s 240ms/step - loss: 1.8005 - accuracy: 0.3863 - val_loss: 1.2670 - val_accuracy: 0.5586

Epoch 00003: saving model to best_model.hdf5
Epoch 4/10
827/827 [==============================] - 197s 238ms/step - loss: 1.7054 - accuracy: 0.4034 - val_loss: 1.2081 - val_accuracy: 0.5677

Epoch 00004: saving model to best_model.hdf5
Epoch 5/10
827/827 [==============================] - 201s 243ms/step - loss: 1.6500 - accuracy: 0.4178 - val_loss: 1.2069 - val_accuracy: 0.5728

Epoch 00005: saving model to best_model.hdf5
Epoch 6/10
827/827 [====================

In [20]:
model2.save('../working/model_last_1.hdf5')
model2.load_weights('best_model.hdf5')

In [21]:
# scores = model2.evaluate_generator(test_generator_2, steps=len(test_generator_2), verbose=1)
# print("Accuracy: %.2f%%" % (scores[1]*100))

In [22]:
# начинаем постепенную разморозку слоев для базовой модели для обучения
base_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = len(base_model.layers)//2 # количество слоев, которые будут обучаться

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

In [23]:
model2.compile(loss="categorical_crossentropy", 
               optimizer=optimizers.Adam(lr=LR), 
               metrics=["accuracy"])

In [24]:
history = model2.fit(
        train_generator_2,
        steps_per_epoch = train_generator_2.samples//train_generator_2.batch_size,
        validation_data = test_generator_2, 
        validation_steps = test_generator_2.samples//test_generator_2.batch_size,
        epochs = EPOCHS,
        callbacks = callbacks_list
)

Epoch 1/10
827/827 [==============================] - 252s 285ms/step - loss: 1.4898 - accuracy: 0.4692 - val_loss: 0.4146 - val_accuracy: 0.8427

Epoch 00001: saving model to best_model.hdf5
Epoch 2/10
827/827 [==============================] - 235s 284ms/step - loss: 0.6233 - accuracy: 0.7794 - val_loss: 0.3239 - val_accuracy: 0.8948

Epoch 00002: saving model to best_model.hdf5
Epoch 3/10
827/827 [==============================] - 232s 280ms/step - loss: 0.4540 - accuracy: 0.8492 - val_loss: 0.2620 - val_accuracy: 0.9164

Epoch 00003: saving model to best_model.hdf5
Epoch 4/10
827/827 [==============================] - 231s 279ms/step - loss: 0.3404 - accuracy: 0.8789 - val_loss: 0.2229 - val_accuracy: 0.9224

Epoch 00004: saving model to best_model.hdf5
Epoch 5/10
827/827 [==============================] - 230s 278ms/step - loss: 0.3077 - accuracy: 0.8931 - val_loss: 0.2589 - val_accuracy: 0.9207

Epoch 00005: saving model to best_model.hdf5
Epoch 6/10
827/827 [====================

In [25]:
base_model.trainable = True

LR = 1e-5
model2.compile(loss="categorical_crossentropy", 
               optimizer=optimizers.Adam(lr=LR), 
               metrics=["accuracy"])

history = model2.fit(
        train_generator_2,
        steps_per_epoch = train_generator_2.samples//train_generator_2.batch_size,
        validation_data = test_generator_2, 
        validation_steps = test_generator_2.samples//test_generator_2.batch_size,
        epochs = EPOCHS,
        callbacks = callbacks_list
)

Epoch 1/10
827/827 [==============================] - 521s 607ms/step - loss: 0.1497 - accuracy: 0.9487 - val_loss: 0.1462 - val_accuracy: 0.9522

Epoch 00001: saving model to best_model.hdf5
Epoch 2/10
827/827 [==============================] - 495s 599ms/step - loss: 0.1418 - accuracy: 0.9484 - val_loss: 0.1402 - val_accuracy: 0.9543

Epoch 00002: saving model to best_model.hdf5
Epoch 3/10
827/827 [==============================] - 498s 601ms/step - loss: 0.1145 - accuracy: 0.9570 - val_loss: 0.1413 - val_accuracy: 0.9565

Epoch 00003: saving model to best_model.hdf5
Epoch 4/10
827/827 [==============================] - 498s 603ms/step - loss: 0.1078 - accuracy: 0.9596 - val_loss: 0.1454 - val_accuracy: 0.9586

Epoch 00004: saving model to best_model.hdf5

Epoch 00004: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-06.
Epoch 5/10
827/827 [==============================] - 500s 605ms/step - loss: 0.1016 - accuracy: 0.9588 - val_loss: 0.1399 - val_accuracy: 0.9599

Epoc

In [26]:
EPOCHS               = 10
BATCH_SIZE           = 4
LR                   = 1e-5

IMG_SIZE             = 456
IMG_CHANNELS         = 3
input_shape          = (IMG_SIZE, int(IMG_SIZE/ratio), IMG_CHANNELS)

In [27]:
AUGMENTATIONS = albumentations.Compose([
    albumentations.HorizontalFlip(p=0.5),
    albumentations.Rotate(limit=30, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=False, p=0.5),

])

# dataloaders
train_datagen_2 = ImageDataAugmentor(
        rescale=1./255,
        augment=AUGMENTATIONS,
        validation_split=VAL_SPLIT)

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [28]:
# data generators
train_generator_2 = train_datagen_2.flow_from_directory(
        PATH+'train/',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True, 
        seed=RANDOM_SEED,
        subset='training')

test_generator_2 = train_datagen_2.flow_from_directory(
        PATH+'train/',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True, 
        seed=RANDOM_SEED,
        subset='validation')

test_sub_generator = test_datagen.flow_from_dataframe( 
    dataframe=sample_submission,
    directory=PATH+'test_upload/',
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,)

Found 13232 images belonging to 10 classes.
Found 2329 images belonging to 10 classes.
Found 6675 validated image filenames.


In [29]:
# загружаем базовую модель

model_path = '../input/efficientnet-keras-noisystudent-weights-b0b7/efficientnet-b5_noisy-student_notop.h5'  # imagenet

base_model = efn.EfficientNetB5(weights=model_path, include_top=False, input_shape=input_shape)

# # Устанавливаем новую "голову" (head)

model2 = Model.Sequential()
model2.add(base_model)
# model2.add(layers.Conv2D(32, 3, activation = 'relu', padding = 'same', 
#                         input_shape = input_shape))
# model2.add(layers.BatchNormalization())

model2.add(layers.GlobalAveragePooling2D())
model2.add(layers.BatchNormalization())
#model2.add(layers.Dropout(0.25))    

model2.add(layers.Dense(256, activation='relu'))
model2.add(layers.Dropout(0.25))
#model2.add(layers.BatchNormalization())
#     model2.add(layers.Dropout(0.25))

model2.add(layers.Dense(CLASS_NUM, activation='softmax'))

model2.compile(loss="categorical_crossentropy", 
               optimizer=optimizers.Adam(lr=LR), 
               metrics=["accuracy"],
               )

In [30]:
# Загружаем веса уже обученной модели
model2.load_weights('best_model.hdf5')

In [31]:
checkpoint = ModelCheckpoint('best_model.hdf5', 
                             monitor = ['val_accuracy'], 
                             verbose = 1, 
                             mode = 'max')

In [32]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3,
                              patience=2, min_lr=0.0000001, verbose=1,
                             mode='auto')
earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                             patience=3, 
                                             restore_best_weights=True)

callbacks_list = [checkpoint,earlystop,reduce_lr]

In [33]:
history = model2.fit(
        train_generator_2,
        steps_per_epoch = train_generator_2.samples//train_generator_2.batch_size,
        validation_data = test_generator_2, 
        validation_steps = test_generator_2.samples//test_generator_2.batch_size,
        epochs = EPOCHS,
        callbacks = callbacks_list
)

Epoch 1/10
3308/3308 [==============================] - 1500s 447ms/step - loss: 0.2208 - accuracy: 0.9278 - val_loss: 0.1150 - val_accuracy: 0.9712

Epoch 00001: saving model to best_model.hdf5
Epoch 2/10
3308/3308 [==============================] - 1483s 448ms/step - loss: 0.1694 - accuracy: 0.9444 - val_loss: 0.1201 - val_accuracy: 0.9699

Epoch 00002: saving model to best_model.hdf5
Epoch 3/10
3308/3308 [==============================] - 1477s 446ms/step - loss: 0.1620 - accuracy: 0.9457 - val_loss: 0.1073 - val_accuracy: 0.9725

Epoch 00003: saving model to best_model.hdf5
Epoch 4/10
3308/3308 [==============================] - 1484s 448ms/step - loss: 0.1541 - accuracy: 0.9502 - val_loss: 0.1158 - val_accuracy: 0.9734

Epoch 00004: saving model to best_model.hdf5
Epoch 5/10
3308/3308 [==============================] - 1480s 447ms/step - loss: 0.1214 - accuracy: 0.9596 - val_loss: 0.1079 - val_accuracy: 0.9725

Epoch 00005: saving model to best_model.hdf5

Epoch 00005: ReduceLROnP

In [34]:
from sklearn.metrics import accuracy_score

In [35]:
test_sub_generator.samples

6675

In [36]:
test_sub_generator.reset()
predictions = model2.predict_generator(test_sub_generator, steps=len(test_sub_generator), verbose=1) 
predictions = np.argmax(predictions, axis=-1) 
label_map = (train_generator_2.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
predictions = [label_map[k] for k in predictions]

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


1669/1669 [==============================] - 124s 73ms/step


In [37]:
filenames_with_dir=test_sub_generator.filenames
submission = pd.DataFrame({'Id':filenames_with_dir, 'Category':predictions}, columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/','')
submission.to_csv('submission.csv', index=False)
print('Save submit')

Save submit


In [38]:
import shutil
shutil.rmtree("/kaggle/working/car/test_upload")
shutil.rmtree("/kaggle/working/car/train")

In [39]:
submission.head()

,Id,Category
0,305108.jpg,7
1,295075.jpg,6
2,31197.jpg,4
3,93598.jpg,7
4,87877.jpg,7


In [40]:
submission.to_csv(PATH+'submission2.csv', index=False)